# **Recomendação de Filmes com Filtragem Colaborativa**

In [2]:
# Instalação e Importação de Bibliotecas

!pip install pandas scikit-learn tensorflow numpy

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.8/620.8 MB 9.9 MB/s eta 0:00:00:00:0100:02


2025-10-29 23:04:44.504660: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-29 23:04:44.560446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 23:04:45.850994: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
# Carregando o Dataset MovieLens 100k

url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
colunas = ["usuario", "filme", "avaliacao", "tempo"]
dados = pd.read_csv(url, sep='\t', names=colunas)

dados.head()

,usuario,filme,avaliacao,tempo
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
# Preparando os Dados

# Criar uma tabela de usuários x filmes
matriz_avaliacoes = dados.pivot_table(index='usuario', columns='filme', values='avaliacao')

# Divide em treino e teste (80% e 20%)
train, test = train_test_split(dados, test_size=0.2, random_state=42)

In [5]:
# Filtragem Colaborativa Baseada em Usuário

# Calcula a similaridade entre os usuários
similaridade_usuarios = cosine_similarity(matriz_avaliacoes.fillna(0))
sim_usuarios_df = pd.DataFrame(
    similaridade_usuarios,
    index=matriz_avaliacoes.index,
    columns=matriz_avaliacoes.index
)

# Função para prever a nota que um usuário daria a um filme
def prever_avaliacao_usuario(usuario, filme):
    # Caso o filme não exista na matriz
    if filme not in matriz_avaliacoes.columns:
        return 3.0  # nota média neutra padrão

    # Pega as avaliações dos usuários para esse filme
    notas_usuario = matriz_avaliacoes[filme]

    # Pega a similaridade do usuário com os outros
    similaridade = sim_usuarios_df[usuario]

    # Considera apenas os usuários que avaliaram o filme
    mask = notas_usuario.notna()

    # Se ninguém avaliou o filme, retorna valor neutro
    if mask.sum() == 0:
        return 3.0

    pred = np.dot(similaridade[mask], notas_usuario[mask]) / similaridade[mask].sum()
    return pred

# Avaliação no conjunto de teste
y_true, y_pred = [], []

for _, row in test.iterrows():
    y_true.append(row['avaliacao'])
    y_pred.append(prever_avaliacao_usuario(row['usuario'], row['filme']))

# Cálculo das métricas
mse_user = mean_squared_error(y_true, y_pred)
rmse_user = np.sqrt(mse_user)
mae_user = mean_absolute_error(y_true, y_pred)

print(f"Filtragem Colaborativa (Usuário) — RMSE: {rmse_user:.3f}, MAE: {mae_user:.3f}")


Filtragem Colaborativa (Usuário) — RMSE: 0.955, MAE: 0.760


In [6]:
# Filtragem Colaborativa Baseada em Itens

# Calcula a similaridade entre os filmes
similaridade_itens = cosine_similarity(matriz_avaliacoes.fillna(0).T)
sim_itens_df = pd.DataFrame(
    similaridade_itens,
    index=matriz_avaliacoes.columns,
    columns=matriz_avaliacoes.columns
)

# Função para prever a nota que um usuário daria a um filme
# com base na similaridade entre os itens
def prever_avaliacao_item(usuario, filme):
    # Caso o filme não exista na matriz
    if filme not in matriz_avaliacoes.columns:
        return 3.0  # nota média neutra padrão

    # Pega todas as notas que o usuário deu
    notas_usuario = matriz_avaliacoes.loc[usuario]

    # Pega a similaridade do filme atual com os outros filmes
    similaridade = sim_itens_df[filme]

    # Considera apenas os filmes avaliados pelo usuário
    mask = notas_usuario.notna()

    # Se o usuário não avaliou nada, retorna nota neutra
    if mask.sum() == 0:
        return 3.0

    pred = np.dot(similaridade[mask], notas_usuario[mask]) / similaridade[mask].sum()
    return pred

# Avaliação no conjunto de teste
y_true, y_pred = [], []

for _, row in test.iterrows():
    y_true.append(row['avaliacao'])
    y_pred.append(prever_avaliacao_item(row['usuario'], row['filme']))

# Cálculo das métricas
mse_item = mean_squared_error(y_true, y_pred)
rmse_item = np.sqrt(mse_item)
mae_item = mean_absolute_error(y_true, y_pred)

print(f"Filtragem Colaborativa (Item) — RMSE: {rmse_item:.3f}, MAE: {mae_item:.3f}")


Filtragem Colaborativa (Item) — RMSE: 0.975, MAE: 0.777


In [7]:
# Autoencoder (Aprendizado Profundo)

# Substitui valores ausentes por 0
matriz_treino = matriz_avaliacoes.fillna(0).values

# Define o modelo de Autoencoder
entrada = tf.keras.layers.Input(shape=(matriz_treino.shape[1],))
codificador = tf.keras.layers.Dense(64, activation='relu')(entrada)
decodificador = tf.keras.layers.Dense(matriz_treino.shape[1], activation='linear')(codificador)

modelo = tf.keras.Model(entrada, decodificador)
modelo.compile(optimizer='adam', loss='mse')

# Treinamento do modelo
modelo.fit(matriz_treino, matriz_treino, epochs=5, batch_size=32, verbose=1)

# Predições e avaliação
predicoes = modelo.predict(matriz_treino)

# Cálculo das métricas
mse_auto = mean_squared_error(matriz_treino.flatten(), predicoes.flatten())
rmse_auto = np.sqrt(mse_auto)
mae_auto = mean_absolute_error(matriz_treino.flatten(), predicoes.flatten())

print(f"Autoencoder — RMSE: {rmse_auto:.3f}, MAE: {mae_auto:.3f}")


Epoch 1/5


2025-10-29 23:05:01.587476: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7724
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6060
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5492
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5150
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4913
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Autoencoder — RMSE: 0.688, MAE: 0.314


In [8]:
# Comparação Final

resultados = pd.DataFrame({
'Método': ['Colab. Usuário', 'Colab. Item', 'Autoencoder'],
'RMSE': [rmse_user, rmse_item, rmse_auto],
'MAE': [mae_user, mae_item, mae_auto]
})
print(resultados)

           Método      RMSE       MAE
0  Colab. Usuário  0.954667  0.759921
1     Colab. Item  0.974673  0.776873
2     Autoencoder  0.688161  0.314141


In [9]:
# PERGUNTA: Qual abordagem foi mais eficiente na recomendação de filmes? Como melhorar o
# sistema de recomendação?

# Qual abordagem foi mais eficiente na recomendação de filmes?
#- O Autoencoder Denso foi a abordagem mais eficiente. Ele apresentou RMSE = 0.680 e MAE = 0.313,
# valores significativamente melhores que os métodos de filtragem colaborativa.
# Isso mostra que suas previsões ficaram mais próximas das avaliações reais dos usuários.

# Como melhorar o sistema de recomendação?
#- Testar com novos dados.
#- Considerar modelos híbridos (autoencoder + filtragem colaborativa).
#- Incorporar metadados de usuários e filmes para enriquecer recomendações.